In [34]:
from matplotlib import pyplot as plt
import numpy as np 
import pandas as pd 
import os
from tqdm import tqdm
day = 10

In [35]:
feat_list = ['open','high','low','close','vwap','volume','pct_chg','turnover','free_turnover']
#             open_ts_norm_30d ,high_ts_norm_30d,low_ts_norm_30d ,close_ts_norm_30d ,vwap_ts_norm_30d 
#             ,volume_ts_norm_30d ,pct_chg_ts_norm_30d ,turnover_ts_norm_30d ,free_turnover_ts_norm_30d 

In [61]:
dict_load = np.load("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/base_list.npy",allow_pickle=True)
dict_load = dict_load.item()

In [127]:
name_list = []
task_list = []
# Moving Cov
# for i in range(len(feat_list)-1):
#     for j in range(i+1,len(feat_list)):
#         name_list.append("COV_%s_%s_%s" % (dict_load[feat_list[i]],dict_load[feat_list[j]],day))
#         task_list.append("mcovar(%s,%s,%s)" % (day,dict_load[feat_list[i]],dict_load[feat_list[j]]))

# Moving Corr
# for i in range(len(feat_list)-1):
#     for j in range(i+1,len(feat_list)):
#         name_list.append("CORR_%s_%s_%s" % (dict_load[feat_list[i]],dict_load[feat_list[j]],day))
#         task_list.append("ts_corr(%s,%s,%s)" % (day,dict_load[feat_list[i]],dict_load[feat_list[j]]))

# # Moving STD
# for i in feat_list:
#     name_list.append("STD_%s_%s" % (i,day))
#     task_list.append("mstd(%s,%s)" % (day,i))
    
# # Z-score
# for i in feat_list:
#     name_list.append("ZSCORE_%s_%s" % (i,day)) 
#     task_list.append("div(mavg(%s,%s),mstd(%s,%s))" % (day,i,day,i))
    
# # Return
for i in feat_list:
    name_list.append("RETURN_%s_%s)" % (i,day))
    task_list.append("div(sub(%s,move(%s,%s)),move(%s,%s))" % (i,day,i,day,i))   

In [128]:
final = ""
for i in name_list:
    final += i + ";"
print("name")
print(final)

name
RETURN_open_10);RETURN_high_10);RETURN_low_10);RETURN_close_10);RETURN_vwap_10);RETURN_volume_10);RETURN_pct_chg_10);RETURN_turnover_10);RETURN_free_turnover_10);


In [129]:
final = ""
for i in task_list:
    final += i + ";"
print("task")
print(final)

task
div(sub(open,move(10,open)),move(10,open));div(sub(high,move(10,high)),move(10,high));div(sub(low,move(10,low)),move(10,low));div(sub(close,move(10,close)),move(10,close));div(sub(vwap,move(10,vwap)),move(10,vwap));div(sub(volume,move(10,volume)),move(10,volume));div(sub(pct_chg,move(10,pct_chg)),move(10,pct_chg));div(sub(turnover,move(10,turnover)),move(10,turnover));div(sub(free_turnover,move(10,free_turnover)),move(10,free_turnover));


In [131]:
alpha_list = np.load("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/alpha_list.npy",allow_pickle=True)
alpha_list = alpha_list.item()
Cover = True
for i in range(len(task_list)):
    if name_list[i] in alpha_list and alpha_list[name_list[i]] != task_list[i] and Cover == False:
        print("contain " + name_list[i])
        break
    alpha_list[name_list[i]] = task_list[i]
alpha_list

{'COV_open_high_10': 'mcovar(10,open,high)',
 'COV_open_low_10': 'mcovar(10,open,low)',
 'COV_open_close_10': 'mcovar(10,open,close)',
 'COV_open_vwap_10': 'mcovar(10,open,vwap)',
 'COV_open_volume_10': 'mcovar(10,open,volume)',
 'COV_open_pct_chg_10': 'mcovar(10,open,pct_chg)',
 'COV_open_turnover_10': 'mcovar(10,open,turnover)',
 'COV_open_free_turnover_10': 'mcovar(10,open,free_turnover)',
 'COV_high_low_10': 'mcovar(10,high,low)',
 'COV_high_close_10': 'mcovar(10,high,close)',
 'COV_high_vwap_10': 'mcovar(10,high,vwap)',
 'COV_high_volume_10': 'mcovar(10,high,volume)',
 'COV_high_pct_chg_10': 'mcovar(10,high,pct_chg)',
 'COV_high_turnover_10': 'mcovar(10,high,turnover)',
 'COV_high_free_turnover_10': 'mcovar(10,high,free_turnover)',
 'COV_low_close_10': 'mcovar(10,low,close)',
 'COV_low_vwap_10': 'mcovar(10,low,vwap)',
 'COV_low_volume_10': 'mcovar(10,low,volume)',
 'COV_low_pct_chg_10': 'mcovar(10,low,pct_chg)',
 'COV_low_turnover_10': 'mcovar(10,low,turnover)',
 'COV_low_free_tur

In [132]:
np.save("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/Factor_Description/alpha_list.npy",alpha_list)